In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [43]:
from datetime import datetime as dt

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 1. Data

In [73]:
# 1. Dataset read
data_path = "datas/movielens_small/{}"

ratings_df = pd.read_csv(data_path.format("ratings.csv"), encoding='utf-8')
ratings_df.drop("timestamp", inplace=True, axis=1)

print("평점 데이터 shape : {}".format(ratings_df.shape))
ratings_df.head()

평점 데이터 shape : (100004, 3)


,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [74]:
movies_df = pd.read_csv(data_path.format("movies.csv"), encoding='utf-8')
movies_df.set_index("movieId", inplace=True)

print("영화 데이터 shape : {}".format(movies_df.shape))
movies_df.head()

영화 데이터 shape : (9125, 2)


,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [75]:
# 2. dummy genre setting
# - 영화별 장르 포함 정보를 가진 dummy data
dummy_genre_df = movies_df['genres'].str.get_dummies(sep="|")

print("* (영화 수, 장르 수)")
print("장르 포함 정보 데이터 shape : {}".format(dummy_genre_df.shape))
dummy_genre_df.head()

* (영화 수, 장르 수)
장르 포함 정보 데이터 shape : (9125, 20)


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [76]:
# 3. year parsing
# - 영화별 년도 정보 추출

# 3-1. (4자리) 타입의 문자열 추출
years = movies_df['title'].str.extract("(\(\d\d\d\d\))")[0].values.astype("str")

# 3-2. 괄호 제거 (nan은 0으로 대치)
years = np.array([0 if y == 'nan' else int(y.replace("(","").replace(")","")) for y in years])

# 3-3. movie column 삽입
movies_df['year'] = years

# 3-4. 년도 정보가 없었던 데이터는 삭제
movies_df.drop(movies_df[movies_df['year'] == 0].index, axis=0, inplace=True)

# 3-5. 제목 칼럼 삭제
movies_df.drop("title", axis=1, inplace=True)

print("영화 데이터 shape : {}".format(movies_df.shape))
movies_df.head()

영화 데이터 shape : (9119, 2)


,genres,year
movieId,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995
2,Adventure|Children|Fantasy,1995
3,Comedy|Romance,1995
4,Comedy|Drama|Romance,1995
5,Comedy,1995


In [77]:
# 4. year-level generate
bins = np.arange(1900, 2023, 20)
labels = [x for x in range(len(bins) - 1)]

# 4-1. 구간 나누기
movies_df['year_level'] = pd.cut(movies_df['year'], bins, right=False, labels=labels)

# 4-2. year 칼럼 삭제
movies_df.drop('year', axis=1, inplace=True)

print("영화 데이터 shape : {}".format(movies_df.shape))
movies_df.head()

영화 데이터 shape : (9119, 2)


,genres,year_level
movieId,,
1,Adventure|Animation|Children|Comedy|Fantasy,4
2,Adventure|Children|Fantasy,4
3,Comedy|Romance,4
4,Comedy|Drama|Romance,4
5,Comedy,4


In [78]:
# 5. threshold data
# - 평점의 갯수가 10개 이상인 영화 데이터 기록

# 5-1. threshold setting each movie
threshold = 10
over_threshold = ratings_df.groupby('movieId').size() >= threshold

# 5-2. rating in
# - 평점의 갯수가 10개 이상인 영화인가? 에 대한 기록
ratings_df['over_threshold'] = ratings_df['movieId'].map(lambda x: over_threshold[x])

# 5-3. filtering
ratings_df = ratings_df[ratings_df['over_threshold']]
ratings_df.drop('over_threshold', axis=1, inplace=True)

print("평점 데이터 shape : {}".format(ratings_df.shape))
ratings_df.head()

평점 데이터 shape : (81915, 3)


,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [104]:
# 6. 최종 학습 데이터 셋 생성
shuffled_df = ratings_df.sample(frac=1)

# rating num user matrix
# 몇 번째 평점의 어떤 유저의 정보인가 matrix
ru_mat = pd.get_dummies(shuffled_df['userId'], prefix="user")

# rating and movie matrix
# 몇 번째 평점의 어떤 영화의 정보인가 matrix
rm_mat = pd.get_dummies(shuffled_df['movieId'], prefix="movie")

# rating and genre matrix
# 몇 번째 평점의 어떤 장르의 정보인가 matrix
rg_mat = shuffled_df['movieId'].apply(lambda x: dummy_genre_df.loc[x])

# ratings and year_level
# 몇 번째 평점의 어떤 년도의 정보인가 vector
ry_vec = shuffled_df['movieId'].apply(lambda x: movies_df.loc[x]['year_level']).rename('year_level')

# concat
concat_df = pd.concat([ru_mat, rm_mat, rg_mat, ry_vec], axis=1)

# 정말 그냥 평점이라는 고유 데이터에
# 사용자 번호 행렬, 영화 번호 행렬, 장르 행렬, 년도 레벨 합친 데이터 이다.
print("최종 학습 데이터셋 shape : {}".format(concat_df.shape))
concat_df.head()

최종 학습 데이터셋 shape : (81915, 2937)


,user_1,user_2,user_3,user_4,user_5,user_6,user_7,user_8,user_9,user_10,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year_level
22508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,5
68373,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
34265,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,5
37254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,4
13470,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [398]:
# 7. 훈련, 시험용 데이터 분리
from sklearn.model_selection import train_test_split

# target_df
## 지금까지와 다른 특징은 평점 데이터를 사용하지 않고
## 4점이 넘으면 보는 것, 넘지 않으면 안 보는 것으로 간주 -> 추천의 관점에서
target_df = ratings_df.loc[concat_df.index]['rating']
target_df = target_df.apply(lambda x: 1 if x >= 4 else 0)

X_train, X_test, y_train, y_test = train_test_split(concat_df, target_df, test_size=0.1)

print("훈련 데이터 shape : {}, 훈련 라벨 갯수 : {}".format(X_train.shape, y_train.size))
print("테스트 데이터 shape : {}, 테스트 라벨 갯수 : {}".format(X_test.shape, y_test.size))

훈련 데이터 shape : (73723, 2937), 훈련 라벨 갯수 : 73723
테스트 데이터 shape : (8192, 2937), 테스트 라벨 갯수 : 8192


In [443]:
# 1. Setting Value 
num_factor = 8
n, p = X_train.shape

In [444]:
# 2. tensor variable ref.Factorization Machine Model Operation (1)

# 2-1. Global Bias
w_0 = tf.Variable([0.0])

# 2-2. i 번째 개별 **특성**에 대한 가중치
w = tf.Variable(tf.zeros(shape=[p]))

# 2-3. v_i, v_factor 는 f개의 latent factor로 표현된 2-way interaction을 계산하는 내적을 의미
v = tf.Variable(tf.random.normal(shape=(p, num_factor)))

trainable_variables = [w_0, w, v]
trainable_variables

[<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(2937,) dtype=float32, numpy=array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(2937, 8) dtype=float32, numpy=
 array([[ 0.4769689 ,  1.2775885 ,  0.5874128 , ..., -2.0577826 ,
          2.0264452 ,  0.04214879],
        [-0.01761444,  0.74643445,  0.02856723, ...,  1.2845016 ,
         -0.9337457 ,  0.9452472 ],
        [-0.25049582, -1.1684519 ,  0.25146428, ...,  0.07809244,
          0.38926372, -0.6039391 ],
        ...,
        [ 0.8807711 ,  0.60043865,  0.24178547, ..., -0.45034876,
         -0.67880976,  2.1861818 ],
        [-0.07838747,  0.47227886,  0.17388366, ...,  0.46838647,
         -1.3519942 , -0.31144962],
        [ 0.26803428, -1.0982381 , -0.5082012 , ...,  0.8455433 ,
          0.2993861 ,  0.75264674]], dtype=float32)>]

In [445]:
model.trainable_variables[0].trainable

True

In [446]:
# 3. predictref.Factorization Machine Model Operation (1)

# 3-1. Setting
batch_size = 32
X_batch = tf.constant(X_train.to_numpy()[:batch_size], dtype=float)

In [497]:
# 3-2. predict function, keras.Model.call function
def predict():
    # 3-1. 시그마(W_i, x_i)
    degree_1 = tf.reduce_sum(tf.multiply(w, X_batch), axis=1)

    # 3-2. 오른쪽에 어려운 식 계산
    degree_2 = 0.5 * tf.reduce_sum(
            tf.math.pow(tf.matmul(X_batch, v), 2)
            - tf.matmul(tf.math.pow(X_batch, 2), tf.math.pow(v, 2)),
            1, keepdims=False
        )

    # 3-3. 나머지 계산 (Sigmoid)
    _predict = tf.math.sigmoid(w_0 + degree_1, degree_2)
    
    return _predict

print("예측 값",predict().numpy())

예측 값 [0.5374253  0.42556968 0.42557    0.40614608 0.43783554 0.48750457
 0.43783462 0.50000054 0.42556968 0.5050036  0.41339773 0.49999925
 0.45017597 0.47253352 0.43046936 0.4725334  0.5049795  0.48001233
 0.43046868 0.48001772 0.4427673  0.41825962 0.470042   0.4013292
 0.44026965 0.50000024 0.524976   0.41097665 0.49250102 0.505003
 0.40614688 0.41825402]


In [514]:
# 4. loss function
loss_fn = keras.losses.binary_crossentropy
with tf.GradientTape() as tape:    
    y_batch = tf.constant(y_train.to_numpy()[:batch_size], dtype=float)
    _predict = predict()
    loss_value = loss_fn(y_batch, _predict)
    print("loss : {}".format(loss_value))
    
# 5. Variable, Weight, Bias Train
learning_rate = 0.01

# 5-1. Backpropagation
gradients = tape.gradient(loss_value, 
              trainable_variables,
              unconnected_gradients=tf.UnconnectedGradients.ZERO)

# 5-2. Optimizer
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

# 5-3. variable train
optimizer.apply_gradients(zip(gradients, trainable_variables))

trainable_variables

loss : 0.44674065709114075


[<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([-0.01986418], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(2937,) dtype=float32, numpy=
 array([ 0.        ,  0.        ,  0.        , ..., -0.19970943,
         0.        , -0.02000255], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(2937, 8) dtype=float32, numpy=
 array([[ 0.4769689 ,  1.2775885 ,  0.5874128 , ..., -2.0577826 ,
          2.0264452 ,  0.04214879],
        [-0.01761444,  0.74643445,  0.02856723, ...,  1.2845016 ,
         -0.9337457 ,  0.9452472 ],
        [-0.25049582, -1.1684519 ,  0.25146428, ...,  0.07809244,
          0.38926372, -0.6039391 ],
        ...,
        [ 0.8807711 ,  0.60043865,  0.24178547, ..., -0.45034876,
         -0.67880976,  2.1861818 ],
        [-0.07838747,  0.47227886,  0.17388366, ...,  0.46838647,
         -1.3519942 , -0.31144962],
        [ 0.26803428, -1.0982381 , -0.5082012 , ...,  0.8455433 ,
          0.2993861 ,  0.75264674]], dtype=float32)>]

In [515]:
class FM(keras.Model):
    def __init__(self, p, n_factor=8, **kwargs):
        super().__init__(**kwargs)
        
        self.w_0 = tf.Variable([0.0])
        self.w = tf.Variable(tf.zeros(shape=[p]))
        self.v = tf.Variable(tf.random.normal(shape=(p, n_factor)))
        
    def call(self, inputs):
        degree_1 = tf.reduce_sum(tf.multiply(self.w, inputs), axis=1)
        
        degree_2 = 0.5 * tf.reduce_sum(
                    tf.math.pow(tf.matmul(inputs, self.v), 2)
                    - tf.matmul(tf.math.pow(inputs, 2), tf.math.pow(self.v, 2)),
                    1, keepdims=False
                )
        
        predict = tf.math.sigmoid(self.w_0 + degree_1, degree_2)
        
        return predict

In [516]:
learner = "adam"
learning_rate = 0.01

if learner == "adagrad":
    optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
elif learner == "rmsprop":
    optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
elif learner == "adam":
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
else:
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

loss_fn = keras.losses.binary_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.BinaryAccuracy()]
test_acc = keras.metrics.BinaryAccuracy()

n, p = X_train.shape

model = FM(p=p)
# model.compile(optimizer=optimizer, loss=loss_fn)

In [517]:
batch_size = 32

train_data = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_train, tf.float32), tf.cast(y_train, tf.float32))).shuffle(500).batch(batch_size)
test_data = tf.data.Dataset.from_tensor_slices(
    (tf.cast(X_test, tf.float32), tf.cast(y_test, tf.float32))).shuffle(200).batch(batch_size)

In [518]:
epochs = 10

n_steps = len(X_train) // batch_size

def print_status_bar(iteration, total, loss, metrics = None):
    metrics = " - ".join([f"{m.name}: {m.result():.4f}"
                          for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print(f"\r{iteration}/{total}  " + metrics ,
          end = end)

for epoch in range(epochs):
    print(f"에포크 : {epoch}/{epochs}")

    for step, (X_batch, y_batch) in enumerate(train_data):
        # train, test data
        with tf.GradientTape() as tape:
            predict = model(X_batch)
            loss = loss_fn(y_batch, predict)
        gradients = tape.gradient(loss, model.trainable_variables,unconnected_gradients=tf.UnconnectedGradients.ZERO)
        result = optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        mean_loss(loss)

        for metric in metrics:
            metric(y_batch, predict)

        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics=metrics)

    for x_test, y_test in test_data:
        prediction = model(x_test)
        test_acc.update_state(y_test, prediction)

    print_status_bar(n_steps * batch_size, n_steps * batch_size, mean_loss, metrics=metrics)
    print("검증 정확도: ", test_acc.result().numpy())
    for metric in [mean_loss] + [test_acc] +metrics:
        metric.reset_states()

에포크 : 0/10
73696/73696  mean: 0.5865 - binary_accuracy: 0.6906
검증 정확도:  0.7158203
에포크 : 1/10
73696/73696  mean: 0.5464 - binary_accuracy: 0.7220
검증 정확도:  0.7158203
에포크 : 2/10
73696/73696  mean: 0.5417 - binary_accuracy: 0.7258
검증 정확도:  0.71435547
에포크 : 3/10
73696/73696  mean: 0.5398 - binary_accuracy: 0.7290
검증 정확도:  0.7188721
에포크 : 4/10
73696/73696  mean: 0.5397 - binary_accuracy: 0.7295
검증 정확도:  0.7213135
에포크 : 5/10
73696/73696  mean: 0.5394 - binary_accuracy: 0.7299
검증 정확도:  0.7161865
에포크 : 6/10
73696/73696  mean: 0.5397 - binary_accuracy: 0.7295
검증 정확도:  0.7167969
에포크 : 7/10
73696/73696  mean: 0.5397 - binary_accuracy: 0.7290
검증 정확도:  0.7182617
에포크 : 8/10
73696/73696  mean: 0.5396 - binary_accuracy: 0.7291
검증 정확도:  0.7167969
에포크 : 9/10
73696/73696  mean: 0.5396 - binary_accuracy: 0.7300
검증 정확도:  0.71850586
